In [240]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.la import EigenValues_Preconditioner
from netgen.occ import *
from netgen.csg import *
from netgen.webgui import Draw as DrawGeo
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import spilu

In [241]:
class Params():
    def __init__(self):
        perfusion_params = {
            "mu": 3.5*0.00750062E-3,
            "kappa": 3.5*0.00750062E-3*5000,
            "pmin":8.0,
            "uin":160.0
        }

        dash_params = {
            "alpha_CO2": 3.27E-5,             # M mmHg^-1
            "alpha_O2": 1.46E-6 ,             # M mmHg^-1
            "K2p": 21.5,                      # M^-1
            "K2pp": 1E-6,                     # M
            "K3p": 11.3,                      # M^-1
            "K3pp": 1E-6,                     # M
            "K5pp": 2.63E-8,                  # M
            "K6pp": 1.91E-8,                  # M
            "pH_rbc": 7.24,                   # Red blood cell pH (with pH_plasma = 7.4).
            "pH_rbc_S": 7.24,
            "pH_pla": 7.4,                    # plasma pH
            "P50_std": 26.8,                  # mmHg
            "P_CO2_std": 40,                  # mmHg
            "alpha": 2.82,                    # parameters for nH calculation
            "beta": 1.20,
            "gamma": 29.25,
            "W_bl": 0.81,                     # nondimensional, con Hct = 0.45
            "beta_O2": 1.46E-6*1E-12,         # mmol/(um3*mmHg)
            "beta_CO2": 3.27E-5*1E-12,        # mmol/(um3*mmHg)
            "Hb_bl": 2.33E-3*1E-12,           # mmol/um3
            "R_rbc": 0.692,                   # nondimensional, con pH_pl = 7.4
            "Hct": 0.45,                      # nondimensional (volumetric rbc fraction in blood)
            "W_rbc": 0.65,                    # nondimensional
            "W_pl": 0.94,                     # nondimensional
            "K1": 7.94E-7*1E-12               # mmol/um3 (hydration constant CO2 + H2O = HCO3 + H)
        }

        transport_params = {
            "d_pla_O2": 1.62E3,               # um2/s
            "d_pla_CO2": 1E3,                 # um2/s
            "d_ba_O2": 0.914E3,               # um2/s
            "d_ba_CO2": 1E3,                  # um2/s
            "h_ba": 0.3,                      # um
            "p_O2_air": 100,                  # mmHg
            "p_CO2_air": 40,                  # mmHg
            "p_O2_in": 40,                    # mmHg
            "p_CO2_in": 45                    # mmHg
        }

        self.p_params = perfusion_params
        self.dash_params = dash_params
        self.t_params = transport_params

In [242]:
class ILU(BaseMatrix):
    # Give a scipy.sparse mat
    def __init__ (self, a):
        super(ILU, self).__init__()
        rows,cols,vals = a.mat.COO()
        A = sp.csr_matrix((vals,(rows,cols))) # NGSolve mat to scipy.sparse
        self.A = A
        self.ilu = spilu(A)
    def Mult (self, x, y):
        x_vec = x.FV().NumPy()
        y_vec = self.ilu.solve(x_vec)
        y.FV()[:] = y_vec
    def Height (self):
        return self.A.shape[0]
    def Width (self):
        return self.A.shape[1]

In [243]:
class NGPerfusionGasExchangeModel():
    def __init__(self, mesh_path, results_path, exp_path):
        self.params = Params()
        self.mesh_path = mesh_path
        self.results_path = os.path.join(results_path, exp_path)

    def prism_setup(self, a, b, h=0.1):

        geo = CSGeometry()
        prism = OrthoBrick(Pnt(*a), Pnt(*b)).bc('air')
        geo.Add(prism)
        ngmesh = geo.GenerateMesh(maxh=h)
        # "left/right"
        # ngmesh.SetBCName(0, 'fluid')
        # ngmesh.SetBCName(4, 'fluid')
        ngmesh.SetBCName(0, 'in')
        ngmesh.SetBCName(4, 'out')
        self.mesh = Mesh(ngmesh)
        Draw(self.mesh, height='3vh')

    # def mesh_setup(self, h=0.1)
        
    def perfusion(self):
        # NGSolve function spaces
        V = VectorH1(self.mesh, order=2)
        P = H1(self.mesh, order=1, dirichlet='out')
        # X = V*P
        (p, v) = P.TnT()

        # Equation
        darcy = InnerProduct(Grad(p), Grad(v))*dx
        rhs = (1/self.params.p_params["kappa"])*self.params.p_params["mu"]*self.params.p_params["uin"]*v.Trace()*ds('in')
        
        a = BilinearForm(darcy).Assemble()
        b = LinearForm(rhs).Assemble()
        
        # gf = GridFunction(X)
        # gfu, gfp = gf.components
        gfu = GridFunction(V)
        gfp = GridFunction(P)

        # Outlet pressure prescribed
        gfp.Set(CF(self.params.p_params["pmin"]), definedon=self.mesh.Boundaries('out'))
        
        res = b.vec - a.mat * gfp.vec
        gfp.vec.data += a.mat.Inverse(freedofs=P.FreeDofs())*res

        # preI = ILU(a)
        
        # solvers.GMRes(A = a.mat,
        #               b = b.vec,
        #             #   pre=preI,
        #               freedofs=P.FreeDofs(),
        #               x = gfp.vec,
        #               tol = 1e-8,
        #               maxsteps=10000,
        #               printrates='\r')

        # solvers.CG(mat = a.mat,
        #         rhs = b.vec,
        #         pre = None, freedofs=P.FreeDofs(),
        #         sol = gfp.vec,
        #         tol = 1e-8,
        #         maxsteps=10000,
        #         printrates='\r')

        # solvers.MinRes(mat = a.mat,
        #         rhs = b.vec,
        #         pre = preI,
        #         sol = gfp.vec,
        #         tol = 1e-8,
        #         maxsteps=10000,
        #         printrates='\r')

        # solvers.QMR(mat = a.mat,
        #         rhs = b.vec,
        #         fdofs=P.FreeDofs(),
        #         sol = gfp.vec,
        #         tol = 1e-8,
        #         maxsteps=10000,
        #         printrates='\r')
        
        gfu.Set(-(1/self.params.p_params['mu'])*self.params.p_params['kappa']*Grad(gfp)) # -grad(p)    
        
        return gfu, gfp
    
    def S_HbXY(self, P_O2, P_CO2):
            '''
            Saturations S_HbO2 and S_HbCO2 as functions of
            gas partial pressures P_O2 and P_CO2. It is assumed
            that partial pressures are equal in plasma and inside
            red blood cells (Dash & Bassingthwaighte, 2010).
            '''

            # Known parameters (reaction rates, Fick's law constants)
            alpha_CO2 = self.params.dash_params["alpha_CO2"]             # M mmHg^-1
            alpha_O2 = self.params.dash_params["alpha_O2"]              # M mmHg^-1

            K2p = self.params.dash_params["K2p"]                      # M^-1
            K2pp = self.params.dash_params["K2pp"]                     # M
            K3p = self.params.dash_params["K3p"]                      # M^-1
            K3pp = self.params.dash_params["K3pp"]                     # M
            K5pp = self.params.dash_params["K5pp"]                  # M
            K6pp = self.params.dash_params["K6pp"]                 # M

            # Red blood cell pH (with pH_plasma = 7.4).
            pH_rbc = self.params.dash_params["pH_rbc"] 
            pH_pla = self.params.dash_params["pH_pla"]

            R_rbc = 10**(-pH_pla+pH_rbc)
            c_H = 10**(-pH_rbc)

            # Variables defined in Dash & Bassingthwaighte (2016).
            phi_1 = 1 + (K2pp/c_H)
            phi_2 = 1 + (K3pp/c_H)
            phi_3 = 1 + (c_H/K5pp)
            phi_4 = 1 + (c_H/K6pp)

            # P50 as function of carbon dioxide deviation from standard.
            P50_std = self.params.dash_params["P50_std"] 
            P_CO2_std = self.params.dash_params["P_CO2_std"] 
            pH_rbc_S = self.params.dash_params["pH_rbc_S"]
            P50_delta_CO2 = P50_std + 1.273E-1*(P_CO2-P_CO2_std) + 1.083E-4*(P_CO2-P_CO2_std)**2
            P50_delta_pH = P50_std - 25.535*(pH_rbc - pH_rbc_S) + 10.646*(pH_rbc - pH_rbc_S)**2 -1.764*(pH_rbc - pH_rbc_S)**3
            P50 = (P50_delta_CO2*P50_delta_pH)/P50_std

            # Variable nH (Hill equation exponent), fit to data.
            alpha = self.params.dash_params["alpha"]
            beta = self.params.dash_params["beta"]
            gamma = self.params.dash_params["gamma"]

            nH = alpha - beta*10**(-P_O2/gamma)
            K4p = ((alpha_O2*P_O2)**(nH-1))*(K2p*alpha_CO2*P_CO2*phi_1 + phi_3)
            K4p /= (((alpha_O2*P50)**(nH))*(K3p*alpha_CO2*P_CO2*phi_2 + phi_4))

            K_HbO2 = (K4p*(K3p*phi_2*alpha_CO2*P_CO2 + phi_4))
            K_HbO2 /= (K2p*phi_1*alpha_CO2*P_CO2 + phi_3)

            K_HbCO2 = (K2p*phi_1 + K3p*K4p*phi_2*alpha_O2*P_O2)
            K_HbCO2 /= (phi_3 + K4p*phi_4*alpha_O2*P_O2)

            S_HbO2 = CF(K_HbO2*alpha_O2*P_O2)/(1+(K_HbO2*alpha_O2*P_O2))
            S_HbCO2 = CF((K_HbCO2*alpha_CO2*P_CO2)/(1+(K_HbCO2*alpha_CO2*P_CO2)))
            
            return S_HbO2, S_HbCO2

    def gas_exchange(self):
    
        ME = H1(self.mesh, order=1, dirichlet='in') * H1(self.mesh, order=1, dirichlet='in')
        
        (p_O2, p_CO2), (v, w) = ME.TnT()

        # Unknown fields
        gf = GridFunction(ME)
        gfp_O2, gfp_CO2 = gf.components
        
        W_bl = self.dash_params["W_bl"]                     # adimensional, con Hct = 0.45
        beta_O2 = self.dash_params["beta_O2"]         # mmol/(um3*mmHg)
        beta_CO2 = self.dash_params["beta_CO2"]        # mmol/(um3*mmHg)
        Hb_bl = self.dash_params["Hb_bl"]           # mmol/um3
        # R_rbc = self.dash_params["R_rbc"]                   # adimensional, con pH_pl = 7.4
        Hct = self.dash_params["Hct"]                      # adimensional (fracción volumétrica de rbcs en sangre)
        W_rbc = self.dash_params["W_rbc"]                    # adimensional
        W_pl = self.dash_params["W_pl"]                    # adimensional
        K1 = self.dash_params["K1"]              # mmol/um3 (constante de hidratación CO2 + H2O = HCO3 + H)
        pH_rbc = self.dash_params["pH_rbc"]                   # log(mmol/um3)
        pH_pla = self.dash_params["pH_pla"]
        c_H = 10**(-pH_rbc)*1E-12       # mmol/um3

        R_rbc = 10**(1.357-0.205*pH_pla)
        bic = ((1-Hct)*W_pl+Hct*W_rbc*R_rbc)*(K1*beta_CO2/c_H)
        # print(f"bic={bic}")
    
        # Saturaciones
        S_HbO2, S_HbCO2 = self.S_HbXY(p_O2.vec, p_CO2.vec)      

        # beta_O2 en mmol/(L*mmHg), p_O2 en mmHg, c_O2 en mmol/L = mM

        # Concentraciones
        c_O2 = W_bl*beta_O2*p_O2 + 4*Hb_bl*S_HbO2
        c_CO2 = W_bl*beta_CO2*p_CO2 + bic*p_CO2 + 4*Hb_bl*S_HbCO2
        # c_O2 = ufl.variable(W_bl*beta_O2*p_O2)
        # c_CO2 = ufl.variable(W_bl*beta_CO2*p_CO2 + bic*p_CO2)

        # if guess is not None:
        #     print("Starting nonlinear problem with previous guess.")
        #     print(f"p_val = {p_val}")
        #     c_O2 += p_val*4*Hb_bl*S_HbO2
        #     c_CO2 += p_val*4*Hb_bl*S_HbCO2

        d_pla_O2 = self.params.t_params["d_pla_O2"]                # um2/s
        d_pla_CO2 = self.params.t_params["d_pla_CO2"]                 # um2/s
        d_ba_O2 = self.params.t_params["d_ba_O2"]               # um2/s
        d_ba_CO2 = self.params.t_params["d_ba_CO2"]                  # um2/s
        h_ba = self.params.t_params["h_ba"]                      # um
        p_O2_air = self.params.t_params["p_O2_air"]                  # mmHg
        p_CO2_air = self.params.t_params["p_CO2_air"]                  # mmHg
        p_O2_in = self.params.t_params["p_O2_in"]                    # mmHg
        p_CO2_in = self.params.t_params["p_CO2_in"]                   # mmHg        
        
        # Oxygen
        F_O2 =  d_pla_O2 * InnerProduct(Grad(c_O2), Grad(v)) * dx               # Dif 1
        F_O2 += (d_ba_O2 / h_ba) * v.Trace() * (beta_O2*p_O2) * ds('air')                    # Dif 2 (variable)
        F_O2 -= (d_ba_O2 / h_ba) * v.Trace() * beta_O2 * p_O2_air * ds('air')       # Dif 2 (constante)

        F_O2 -= InnerProduct(self.u * c_O2, Grad(v)) * dx                            # Adv
        F_O2 += InnerProduct(self.u * c_O2, specialcf.normal(self.mesh.dim)) * v.Trace() * ds('out')

        # Carbon dioxide
        F_CO2 =  d_pla_CO2 * InnerProduct(Grad(c_CO2), Grad(w)) * dx              # Dif 1
        F_CO2 += (d_ba_CO2 / h_ba) * w.Trace() * (beta_CO2*p_CO2) * ds('air')                    # Dif 2 (variable)
        F_CO2 -= (d_ba_CO2 / h_ba) * w.Trace() * beta_CO2 * p_CO2_air * ds('air')       # Dif 2 (constante)

        F_CO2 -= InnerProduct(self.u * c_CO2, Grad(w)) * dx                            # Adv
        F_CO2 += InnerProduct(self.u * c_CO2, specialcf.normal(self.mesh.dim)) * w.Trace() * ds('out')

        # Complete functional
        F = F_O2 + F_CO2

        a = BilinearForm(ME)
        a += Variation(F.Compile()*dx)

        gfp_O2.Set(CF(p_O2_in), definedon=self.mesh.Boundaries('in'))
        gfp_CO2.Set(CF(p_CO2_in), definedon=self.mesh.Boundaries('in'))

        res_O2 = - a.mat * gfp_O2.vec
        res_CO2 = - a.mat * gfp_CO2.vec

        solvers.GMRes(A =a.mat, 
                      b = res_O2 + res_CO2, 
                      pre=None, freedofs=ME.FreeDofs(), 
                      x =gf.vec, 
                      tol=1e-8,
                      maxsteps=10000,
                      printrates='\r')  

In [244]:
ngmodel = NGPerfusionGasExchangeModel('', '', '')
ngmodel.prism_setup((0,0,0), (8,8,200), 2)

WebGuiWidget(layout=Layout(height='3vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.230…

In [245]:
gfu, gfp = ngmodel.perfusion()

In [247]:
Draw(gfu, height='3vh')

WebGuiWidget(layout=Layout(height='3vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.230…

BaseWebGuiScene